In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
import pickle as pkl
import eipy.deep_ei as d
from scikeras.wrappers import KerasClassifier
import sys

2023-12-07 17:12:46.549155: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-07 17:12:46.579360: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 17:12:46.579403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 17:12:46.580232: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 17:12:46.585476: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-07 17:12:46.586068: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [8]:
from eipy.metrics import fmax_score, fmax_precision_recall_threshold
from sklearn.metrics import roc_auc_score, matthews_corrcoef

metric_funs = {
            'f_max': fmax_score,
            'auc': roc_auc_score,
            'mcc': matthews_corrcoef
            }

In [9]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [10]:
inanimate_labels=[0,1,8,9]
def binarize_array(arr):
    mask = np.isin(arr, inanimate_labels)

    arr = np.where(mask, 0, 1)

    return arr

In [11]:
y_train = binarize_array(y_train)
y_test = binarize_array(y_test)

In [12]:

# with open("/home/opc/block_vol/cifar_10_resized_224/X_train.pkl", "rb") as file:
#     X_train_resized=pkl.load(file=file)
# file.close()
# with open("/home/opc/block_vol/cifar_10_resized_224/X_test.pkl", "rb") as file:
#     X_test_resized=pkl.load(file=file)
# file.close()

In [13]:
from skimage.transform import resize
new_size = (224, 224)

X_train_resized = np.empty((X_train.shape[0], new_size[0], new_size[1], 3))
X_test_resized = np.empty((X_test.shape[0], new_size[0], new_size[1], 3))

for i in range(X_train.shape[0]):
    X_train_resized[i] = resize(X_train[i], new_size, preserve_range=True, mode='reflect')
for i in range(X_test.shape[0]):
    X_test_resized[i] = resize(X_test[i], new_size, preserve_range=True, mode='reflect')

In [14]:
X_sample = X_train_resized[0:5000]
X_sample.shape

(5000, 224, 224, 3)

In [15]:
y_sample = y_train[0:5000]
y_sample.shape
pd.Series(y_sample.flatten()).value_counts()

1    3017
0    1983
Name: count, dtype: int64

In [16]:
X_test_sample = X_test[0:1000]
y_test_sample = y_test[0:1000]

X_test_sample.shape, pd.Series(y_test_sample.flatten()).value_counts()

((1000, 32, 32, 3),
 1    593
 0    407
 Name: count, dtype: int64)

In [17]:
# with open("/home/opc/block_vol/cifar_10_resized_224/X_train.pkl", "wb") as file:
#     pkl.dump(file=file, obj=X_train_resized)
# with open("/home/opc/block_vol/cifar_10_resized_224/X_test.pkl", "wb") as file:
#     pkl.dump(file=file, obj=X_test_resized)

In [ ]:
####HI OVER HERE###
import importlib
importlib.reload(d)
###HEYOOOOOO#

In [ ]:
# with open("/home/opc/eipy/eipy/res_output_weights.pkl", "wb") as file:
#     pkl.dump(res_model.layers[-1].weights, file=file)

In [ ]:
from keras import layers, models
from keras.applications import ResNet50
res_model = ResNet50(weights='imagenet', include_top=False)
for layer in res_model.layers:
    layer.trainable = False

with open("/home/opc/eipy/eipy/bp_initial_output_weights/res_output_weights.pkl","rb") as file:
    res_output_weights = pkl.load(file=file)

tf.random.set_seed(42)
collecting_layer = layers.GlobalAveragePooling2D()(res_model.output)
output_layer = layers.Dense(1, activation='sigmoid', weights=res_output_weights)(collecting_layer)

res_model = models.Model(inputs=res_model.input, outputs=output_layer)
res_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# net_output_weights=net_model.layers[-1].weights
# with open("/home/opc/eipy/eipy/net_output_weights.pkl", "wb") as file:
#     pkl.dump(net_output_weights, file=file)

In [ ]:
from keras.applications.mobilenet import MobileNet
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

base_model = MobileNet(weights='imagenet', include_top=False)

with open("/home/opc/eipy/eipy/bp_initial_output_weights/net_output_weights.pkl", "rb") as file:
    net_output_weights = pkl.load(file=file)

x = base_model.output
for layer in base_model.layers:
    layer.trainable = False
tf.random.set_seed(42)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid', weights=net_output_weights)(x)
net_model = Model(inputs=base_model.input, outputs=predictions)

net_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# v3_output_weights = v3_model.layers[-1].weights
# with open("/home/opc/eipy/eipy/v3_output_weights.pkl", "wb") as file:
#     pkl.dump(v3_output_weights, file=file)

In [ ]:
from keras import layers, models
from keras.applications import InceptionV3

pretrained_model = InceptionV3(weights='imagenet', include_top=False)

with open("/home/opc/eipy/eipy/bp_initial_output_weights/v3_output_weights.pkl", "rb") as file:
    v3_output_weights = pkl.load(file=file)

for layer in pretrained_model.layers:
    layer.trainable = False

collecting_layer = layers.GlobalAveragePooling2D()(pretrained_model.output)
output_layer = layers.Dense(1, activation='sigmoid', weights=v3_output_weights)(collecting_layer)

v3_model = models.Model(inputs=pretrained_model.input, outputs=output_layer)
v3_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# dense_output_weights = dense_model.layers[-1].weights
# with open("/home/opc/eipy/eipy/dense_output_weights.pkl", "wb") as file:
#     pkl.dump(dense_output_weights, file=file)

In [ ]:
from keras import layers, models
from keras.applications import DenseNet121

pretrained_model = DenseNet121(weights='imagenet', include_top=False)

for layer in pretrained_model.layers:
    layer.trainable = False

with open("/home/opc/eipy/eipy/bp_initial_output_weights/dense_output_weights.pkl", "rb") as file:
    dense_output_weights = pkl.load(file=file)

collecting_layer = layers.GlobalAveragePooling2D()(pretrained_model.output)

output_layer = layers.Dense(1, activation='sigmoid', weights=dense_output_weights)(collecting_layer)

dense_model = models.Model(inputs=pretrained_model.input, outputs=output_layer)
dense_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
base_predictors = {
    # "ResNet50" : res_model,
    "MobileNet" : net_model,
    # "InceptionV3" : v3_model,
    "Dense121" : dense_model
}

In [ ]:
EI = d.EnsembleIntegration(
                        k_outer=2,
                        k_inner=2,
                        n_samples=1,
                        sampling_aggregation=None,
                        sampling_strategy=None,
                        n_jobs=-1,
                        metrics=metric_funs,
                        random_state=38,
                        project_name="images",
                        parallel_backend="multiprocessing",
                        model_building=True,
                        )

In [76]:
EI.fit_base(X_sample, y_sample, modality_name="images", base_predictors=base_predictors)

In [ ]:
EI.base_summary["metrics"]

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import pandas as pd
from eipy.ei import EnsembleIntegration
from eipy.additional_ensembles import MeanAggregation, CES
ensemble_predictors = {
                    'Mean' : MeanAggregation(),
                    'CES' : CES(scoring=lambda y_test, y_pred: fmax_score(y_test, y_pred)[0]),
                    'S.ADAB': AdaBoostClassifier(),
                    'S.XGB': XGBClassifier(),
                    'S.DT': DecisionTreeClassifier(),
                    "S.RF": RandomForestClassifier(),
                    'S.GB': GradientBoostingClassifier(),
                    'S.KNN': KNeighborsClassifier(),
                    'S.LR': LogisticRegression(),
                    'S.NB': GaussianNB(),
                    'S.MLP': MLPClassifier(),
                    'S.SVM': SVC(probability=True),
}
EI.fit_ensemble(ensemble_predictors=ensemble_predictors)

In [ ]:
EI.ensemble_summary["metrics"]

In [ ]:
X_dict = {"images": X_test_sample}

In [ ]:
def model_key(metric):
    return EI.ensemble_summary["metrics"].loc[f"{metric}"].idxmax()

In [ ]:
metrics = EI.ensemble_summary["metrics"].index.tolist()
y_preds = {}
for metric in metrics:
    y_preds[metric] = EI.predict(X_dict=X_dict, ensemble_model_key=model_key(metric))

In [ ]:
y_test_sample

In [ ]:
# results={}
# for k,v in y_preds.items():
#     results[k]= metric_funs[k](y_test, v)

In [ ]:
roc_auc_score(y_test_sample, y_preds["auc"])

In [ ]:
fmax_score(y_test_sample, y_preds["f_max"])

In [ ]:
y_preds_mcc = [y>0.5 for y in y_preds["mcc"]]
matthews_corrcoef(y_test_sample, y_preds_mcc)